## 1. Importação das Bibliotecas Necessárias



In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.metrics import silhouette_score
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_score, recall_score, f1_score, roc_curve,
    precision_recall_curve, average_precision_score, accuracy_score
)

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

from scipy import stats
from scipy.stats import wilcoxon, ttest_rel
import time
from collections import Counter

In [ ]:
RANDOM_SEED = 33
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## 2. Carregamento dos dados

In [ ]:
data_set = '/content/sample_data/creditcard.csv'
df = pd.read_csv(data_set)

2.1 Dimensão dos dados

In [ ]:
print(f"Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")

2.2 Tipos de dados


In [ ]:
print("\TIPOS DE DADOS:")
print(df.dtypes)

2.3 Primeiras linhas

In [ ]:
print("\n PRIMEIRAS 5 LINHAS:")
display(df.head())

2.4 Estatíticas descritivas dos dados

In [ ]:
print("\n ESTATÍSTICAS DESCRITIVAS:")
display(df.describe())

2.5 Remoção de duplicatas

In [ ]:
initial_len = df.shape[0]
df = df.drop_duplicates()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartadas {initial_len - df.shape[0]} duplicadas')

2.6 Missing values

In [ ]:
initial_len = df.shape[0]
df = df.dropna()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartados {initial_len - df.shape[0]} registros com valores NA')

2.7 Distribuição de classes

In [ ]:
print(f"\nDistribuição da Classe Target (Class):")
print(df['Class'].value_counts())
print(f"\n   Proporção de Fraudes: {df['Class'].mean()*100:.4f}%")

# 3. Análise Exploratória de Dados

3.1 Análise do Desbalanceamento de Classes

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

class_counts = df['Class'].value_counts()
colors = ['#2ecc71', '#e74c3c']
bars = axes[0].bar(['Normal (0)', 'Fraude (1)'], class_counts.values, color=colors, edgecolor='black')
axes[0].set_title('Distribuição das Classes', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Número de Transações', fontsize=12)
axes[0].set_xlabel('Classe', fontsize=12)

for bar, count in zip(bars, class_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                 f'{count:,}', ha='center', va='bottom', fontsize=11, fontweight='bold')

explode = (0, 0.1)
axes[1].pie(class_counts.values, explode=explode, labels=['Normal', 'Fraude'],
            colors=colors, autopct='%1.3f%%', shadow=True, startangle=90,
            textprops={'fontsize': 12})
axes[1].set_title('Proporção das Classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("ANÁLISE DO DESBALANCEAMENTO")
fraud_count = df[df['Class'] == 1].shape[0]
normal_count = df[df['Class'] == 0].shape[0]
print(f"\nTransações Normais: {normal_count:,} ({normal_count/len(df)*100:.3f}%)")
print(f"Transações Fraudulentas: {fraud_count:,} ({fraud_count/len(df)*100:.3f}%)")
print(f"Razão de Desbalanceamento: 1:{normal_count//fraud_count}")

 3.2 Análise das Features Time e Amount

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

fraud_df = df[df['Class'] == 1]
normal_df = df[df['Class'] == 0]

axes[0, 0].hist(normal_df['Time'], bins=50, alpha=0.7, label='Normal', color='#2ecc71', density=True)
axes[0, 0].hist(fraud_df['Time'], bins=50, alpha=0.7, label='Fraude', color='#e74c3c', density=True)
axes[0, 0].set_title('Distribuição de Time por Classe', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Time (segundos)')
axes[0, 0].set_ylabel('Densidade')
axes[0, 0].legend()

axes[0, 1].hist(normal_df['Amount'], bins=50, alpha=0.7, label='Normal', color='#2ecc71', density=True)
axes[0, 1].hist(fraud_df['Amount'], bins=50, alpha=0.7, label='Fraude', color='#e74c3c', density=True)
axes[0, 1].set_title('Distribuição de Amount por Classe', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Amount ($)')
axes[0, 1].set_ylabel('Densidade')
axes[0, 1].legend()
axes[0, 1].set_xlim(0, 2000)

df.boxplot(column='Amount', by='Class', ax=axes[1, 0])
axes[1, 0].set_title('Boxplot de Amount por Classe', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Classe')
axes[1, 0].set_ylabel('Amount ($)')
axes[1, 0].set_ylim(0, 500)
plt.suptitle('')

df['Hour'] = (df['Time'] / 3600) % 24
fraud_hours = df[df['Class'] == 1]['Hour']
normal_hours = df[df['Class'] == 0]['Hour']

axes[1, 1].hist(normal_hours, bins=24, alpha=0.7, label='Normal', color='#2ecc71', density=True)
axes[1, 1].hist(fraud_hours, bins=24, alpha=0.7, label='Fraude', color='#e74c3c', density=True)
axes[1, 1].set_title('Distribuição por Hora do Dia', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Hora do Dia')
axes[1, 1].set_ylabel('Densidade')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('time_amount_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("ESTATÍSTICAS DE TIME E AMOUNT")
print("\nAmount - Transações Normais:")
print(f"   Média: ${normal_df['Amount'].mean():.2f}")
print(f"   Mediana: ${normal_df['Amount'].median():.2f}")
print(f"   Desvio Padrão: ${normal_df['Amount'].std():.2f}")
print(f"   Máximo: ${normal_df['Amount'].max():.2f}")

print("\nAmount - Transações Fraudulentas:")
print(f"   Média: ${fraud_df['Amount'].mean():.2f}")
print(f"   Mediana: ${fraud_df['Amount'].median():.2f}")
print(f"   Desvio Padrão: ${fraud_df['Amount'].std():.2f}")
print(f"   Máximo: ${fraud_df['Amount'].max():.2f}")

3.3 Análise das Features PCA (V1-V28)

In [ ]:
v_features = [f'V{i}' for i in range(1, 29)]

fig, axes = plt.subplots(4, 4, figsize=(20, 16))
axes = axes.flatten()

for i, feature in enumerate(v_features[:16]):
    axes[i].hist(normal_df[feature], bins=50, alpha=0.6, label='Normal', color='#2ecc71', density=True)
    axes[i].hist(fraud_df[feature], bins=50, alpha=0.6, label='Fraude', color='#e74c3c', density=True)
    axes[i].set_title(f'Distribuição de {feature}', fontsize=10)
    axes[i].legend(fontsize=8)

plt.tight_layout()
plt.savefig('v_features_distribution_1.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(20, 16))
axes = axes.flatten()

for i, feature in enumerate(v_features[12:28]):
    axes[i].hist(normal_df[feature], bins=50, alpha=0.6, label='Normal', color='#2ecc71', density=True)
    axes[i].hist(fraud_df[feature], bins=50, alpha=0.6, label='Fraude', color='#e74c3c', density=True)
    axes[i].set_title(f'Distribuição de {feature}', fontsize=10)
    axes[i].legend(fontsize=8)

plt.tight_layout()
plt.savefig('v_features_distribution_2.png', dpi=150, bbox_inches='tight')
plt.show()

3.4 Matriz de Correlação

In [ ]:
plt.figure(figsize=(24, 20))
correlation_matrix = df.corr()

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='RdBu_r',
            center=0, linewidths=0.5, fmt='.2f',
            cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlação das Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("CORRELAÇÃO DAS FEATURES COM A CLASSE TARGET")
correlations_with_class = df.corr()['Class'].drop('Class').sort_values(key=abs, ascending=False)
print("\nTop 10 correlações mais fortes (em valor absoluto):")
print(correlations_with_class.head(10))
print("\nTop 10 correlações mais fracas:")
print(correlations_with_class.tail(10))

# 4. Pré-processamento de dados


4.1 Normalização das features Time e Amount

In [ ]:
df_processed = df.copy()

if 'Hour' in df_processed.columns:
    df_processed = df_processed.drop('Hour', axis=1)

robust_scaler = RobustScaler()
standard_scaler = StandardScaler()

df_processed['Amount_scaled'] = robust_scaler.fit_transform(df_processed['Amount'].values.reshape(-1, 1))

df_processed['Time_scaled'] = standard_scaler.fit_transform(df_processed['Time'].values.reshape(-1, 1))

df_processed = df_processed.drop(['Time', 'Amount'], axis=1)

print("Normalização concluída!")
print(f"\nShape do dataset processado: {df_processed.shape}")
print(f"\nColunas após pré-processamento:")
print(df_processed.columns.tolist())

4.2 Separação dos Dados em Treino, Validação e Teste

Utilizaremos a seguinte divisão:
*   Treino: 70% dos dados
*   Validação: 15% dos dados
*   Teste: 15% dos dados

Para detecção de anomalias em abordagem não supervisionada, treinaremos os modelos apenas com transações normais.

In [ ]:
X = df_processed.drop('Class', axis=1)
y = df_processed['Class']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_SEED, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=RANDOM_SEED, stratify=y_temp
)

print("DIVISÃO DOS DADOS")

print(f"\nConjunto de TREINO:")
print(f"   Total: {len(X_train):,} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Normal: {(y_train == 0).sum():,}")
print(f"   Fraude: {(y_train == 1).sum():,}")

print(f"\nConjunto de VALIDAÇÃO:")
print(f"   Total: {len(X_val):,} amostras ({len(X_val)/len(X)*100:.1f}%)")
print(f"   Normal: {(y_val == 0).sum():,}")
print(f"   Fraude: {(y_val == 1).sum():,}")

print(f"\nConjunto de TESTE:")
print(f"   Total: {len(X_test):,} amostras ({len(X_test)/len(X)*100:.1f}%)")
print(f"   Normal: {(y_test == 0).sum():,}")
print(f"   Fraude: {(y_test == 1).sum():,}")

In [ ]:
X_train_normal = X_train[y_train == 0]
y_train_normal = y_train[y_train == 0]


print("CONJUNTO DE TREINO PARA ABORDAGEM NÃO SUPERVISIONADA")

print(f"\nTreino apenas com transações NORMAIS:")
print(f"   Total: {len(X_train_normal):,} amostras")
print(f"   (Usaremos este conjunto para treinar os modelos de anomalia)")

# 5. Modelagem

5.1 Cálculo de métricas

In [ ]:
def evaluate_model(y_true, y_pred, y_scores=None, model_name="Model"):
    metrics = {
        'Model': model_name,
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, zero_division=0),
    }

    if y_scores is not None:
        metrics['AUC-ROC'] = roc_auc_score(y_true, y_scores)
        metrics['AUC-PR'] = average_precision_score(y_true, y_scores)

    return metrics


def plot_confusion_matrix(y_true, y_pred, model_name="Model", ax=None):
    cm = confusion_matrix(y_true, y_pred)

    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Normal', 'Fraude'],
                yticklabels=['Normal', 'Fraude'])
    ax.set_xlabel('Predito')
    ax.set_ylabel('Real')
    ax.set_title(f'Matriz de Confusão - {model_name}')

    return cm


def print_metrics(metrics):
    print(f"MÉTRICAS - {metrics['Model']}")
    print(f"  Precisão:  {metrics['Precision']:.4f}")
    print(f"  Recall:    {metrics['Recall']:.4f}")
    print(f"  F1-Score:  {metrics['F1-Score']:.4f}")
    if 'AUC-ROC' in metrics:
        print(f"  AUC-ROC:   {metrics['AUC-ROC']:.4f}")
    if 'AUC-PR' in metrics:
        print(f"  AUC-PR:    {metrics['AUC-PR']:.4f}")

all_results = {}
all_predictions = {}
all_scores = {}
all_times = {}

5.2 Isolation Forest

In [ ]:
contamination_rate = y_val.mean()
print(f"Taxa de contaminação esperada: {contamination_rate:.4f} ({contamination_rate*100:.2f}%)")

print("\n" + "=" * 60)
print("TREINAMENTO - ISOLATION FOREST")
print("=" * 60)

if_params = {
    'n_estimators': 100,
    'contamination': contamination_rate,
    'max_samples': 'auto',
    'random_state': 42,
    'n_jobs': -1
}

start_time = time.time()
iso_forest = IsolationForest(**if_params)
iso_forest.fit(X_train_normal)
train_time_if = time.time() - start_time

print(f"Modelo treinado em {train_time_if:.2f} segundos")

start_time = time.time()
if_val_pred_raw = iso_forest.predict(X_val)
if_val_pred = np.where(if_val_pred_raw == -1, 1, 0)
if_val_scores = -iso_forest.score_samples(X_val)
inference_time_if_val = time.time() - start_time

print(f"Tempo de inferência (validação): {inference_time_if_val:.2f} segundos")

if_val_metrics = evaluate_model(y_val, if_val_pred, if_val_scores, "Isolation Forest")
print_metrics(if_val_metrics)

plt.figure(figsize=(8, 6))
plot_confusion_matrix(y_val, if_val_pred, "Isolation Forest (Validação)")
plt.tight_layout()
plt.savefig('confusion_matrix_if_val.png', dpi=150, bbox_inches='tight')
plt.show()

5.2.1 Tunagem de hiperparâmetros

In [ ]:
print("TUNAGEM DE HIPERPARÂMETROS - ISOLATION FOREST")

n_estimators_list = [50, 100, 200]
max_samples_list = [256, 512, 'auto']
contamination_list = [0.001, 0.002, 0.005, contamination_rate]

best_if_score = 0
best_if_params = {}
if_tuning_results = []

for n_est in n_estimators_list:
    for max_samp in max_samples_list:
        for contam in contamination_list:
            try:
                model = IsolationForest(
                    n_estimators=n_est,
                    max_samples=max_samp,
                    contamination=contam,
                    random_state=42,
                    n_jobs=-1
                )
                model.fit(X_train_normal)

                pred_raw = model.predict(X_val)
                pred = np.where(pred_raw == -1, 1, 0)
                scores = -model.score_samples(X_val)

                f1 = f1_score(y_val, pred, zero_division=0)
                auc = roc_auc_score(y_val, scores)

                if_tuning_results.append({
                    'n_estimators': n_est,
                    'max_samples': max_samp,
                    'contamination': contam,
                    'F1-Score': f1,
                    'AUC-ROC': auc
                })

                if f1 > best_if_score:
                    best_if_score = f1
                    best_if_params = {
                        'n_estimators': n_est,
                        'max_samples': max_samp,
                        'contamination': contam
                    }
            except Exception as e:
                print(f"Erro com params {n_est}, {max_samp}, {contam}: {e}")

if_tuning_df = pd.DataFrame(if_tuning_results).sort_values('F1-Score', ascending=False)
print("\nTop 10 combinações de hiperparâmetros:")
print(if_tuning_df.head(10).to_string(index=False))

print(f"\nMelhores hiperparâmetros encontrados:")
print(f"   n_estimators: {best_if_params['n_estimators']}")
print(f"   max_samples: {best_if_params['max_samples']}")
print(f"   contamination: {best_if_params['contamination']}")
print(f"   F1-Score: {best_if_score:.4f}")

5.2.3 Modelo final

In [ ]:
print("MODELO FINAL - ISOLATION FOREST")

iso_forest_best = IsolationForest(
    n_estimators=best_if_params['n_estimators'],
    max_samples=best_if_params['max_samples'],
    contamination=best_if_params['contamination'],
    random_state=42,
    n_jobs=-1
)

start_time = time.time()
iso_forest_best.fit(X_train_normal)
train_time_if_best = time.time() - start_time

start_time = time.time()
if_test_pred_raw = iso_forest_best.predict(X_test)
if_test_pred = np.where(if_test_pred_raw == -1, 1, 0)
if_test_scores = -iso_forest_best.score_samples(X_test)
inference_time_if = time.time() - start_time

if_test_metrics = evaluate_model(y_test, if_test_pred, if_test_scores, "Isolation Forest (Best)")
print_metrics(if_test_metrics)
print(f"\nTempo de treino: {train_time_if_best:.2f}s")
print(f"Tempo de inferência (teste): {inference_time_if:.4f}s")
print(f"Tempo médio por amostra: {inference_time_if/len(X_test)*1000:.4f}ms")

all_results['Isolation Forest'] = if_test_metrics
all_predictions['Isolation Forest'] = if_test_pred
all_scores['Isolation Forest'] = if_test_scores
all_times['Isolation Forest'] = {
    'train': train_time_if_best,
    'inference': inference_time_if,
    'per_sample': inference_time_if/len(X_test)*1000
}

5.3 Local Outlier Factor

In [ ]:
lof_params = {
    'n_neighbors': 20,
    'contamination': contamination_rate,
    'novelty': True,
    'n_jobs': -1
}

start_time = time.time()
lof = LocalOutlierFactor(**lof_params)
lof.fit(X_train_normal)
train_time_lof = time.time() - start_time

print(f"Modelo treinado em {train_time_lof:.2f} segundos")

start_time = time.time()
lof_val_pred_raw = lof.predict(X_val)
lof_val_pred = np.where(lof_val_pred_raw == -1, 1, 0)
lof_val_scores = -lof.score_samples(X_val)
inference_time_lof_val = time.time() - start_time

print(f"Tempo de inferência (validação): {inference_time_lof_val:.2f} segundos")

lof_val_metrics = evaluate_model(y_val, lof_val_pred, lof_val_scores, "LOF")
print_metrics(lof_val_metrics)

plt.figure(figsize=(8, 6))
plot_confusion_matrix(y_val, lof_val_pred, "LOF (Validação)")
plt.tight_layout()
plt.savefig('confusion_matrix_lof_val.png', dpi=150, bbox_inches='tight')
plt.show()

5.3.1 Tunagem de hiperparâmetros

In [ ]:
n_neighbors_list = [5, 10, 20, 30, 50]
contamination_list_lof = [0.001, 0.002, 0.005, contamination_rate]

best_lof_score = 0
best_lof_params = {}
lof_tuning_results = []

for n_neigh in n_neighbors_list:
    for contam in contamination_list_lof:
        try:
            model = LocalOutlierFactor(
                n_neighbors=n_neigh,
                contamination=contam,
                novelty=True,
                n_jobs=-1
            )
            model.fit(X_train_normal)

            pred_raw = model.predict(X_val)
            pred = np.where(pred_raw == -1, 1, 0)
            scores = -model.score_samples(X_val)

            f1 = f1_score(y_val, pred, zero_division=0)
            auc = roc_auc_score(y_val, scores)

            lof_tuning_results.append({
                'n_neighbors': n_neigh,
                'contamination': contam,
                'F1-Score': f1,
                'AUC-ROC': auc
            })

            if f1 > best_lof_score:
                best_lof_score = f1
                best_lof_params = {
                    'n_neighbors': n_neigh,
                    'contamination': contam
                }
        except Exception as e:
            print(f"Erro com params {n_neigh}, {contam}: {e}")

lof_tuning_df = pd.DataFrame(lof_tuning_results).sort_values('F1-Score', ascending=False)
print("\nTop 10 combinações de hiperparâmetros:")
print(lof_tuning_df.head(10).to_string(index=False))

print(f"\nMelhores hiperparâmetros encontrados:")
print(f"   n_neighbors: {best_lof_params['n_neighbors']}")
print(f"   contamination: {best_lof_params['contamination']}")
print(f"   F1-Score: {best_lof_score:.4f}")

In [ ]:
print("MODELO FINAL - LOF")

lof_best = LocalOutlierFactor(
    n_neighbors=best_lof_params['n_neighbors'],
    contamination=best_lof_params['contamination'],
    novelty=True,
    n_jobs=-1
)

start_time = time.time()
lof_best.fit(X_train_normal)
train_time_lof_best = time.time() - start_time

start_time = time.time()
lof_test_pred_raw = lof_best.predict(X_test)
lof_test_pred = np.where(lof_test_pred_raw == -1, 1, 0)
lof_test_scores = -lof_best.score_samples(X_test)
inference_time_lof = time.time() - start_time

lof_test_metrics = evaluate_model(y_test, lof_test_pred, lof_test_scores, "LOF (Best)")
print_metrics(lof_test_metrics)
print(f"\nTempo de treino: {train_time_lof_best:.2f}s")
print(f"Tempo de inferência (teste): {inference_time_lof:.4f}s")
print(f"Tempo médio por amostra: {inference_time_lof/len(X_test)*1000:.4f}ms")

all_results['LOF'] = lof_test_metrics
all_predictions['LOF'] = lof_test_pred
all_scores['LOF'] = lof_test_scores
all_times['LOF'] = {
    'train': train_time_lof_best,
    'inference': inference_time_lof,
    'per_sample': inference_time_lof/len(X_test)*1000
}